## License Plate Detection using Fuzzy Join 

### Start EVA server

We are reusing the start server notebook for launching the EVA server.

In [3]:
# !wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Adding the images to EVADB for analysis

In [7]:
cursor.execute('DROP TABLE IF EXISTS MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "../data/license/Cars0.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "../data/license/Cars1.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "../data/license/Cars2.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "../data/license/Cars3.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "../data/license/Cars4.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "../data/license/Cars5.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "../data/license/Cars6.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
 Empty DataFrame
Columns: []
Index: []
@query_time: 0.014810735359787941
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded IMAGE: 1
@query_time: 0.0974723007529974
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded IMAGE: 1
@query_time: 0.02928401343524456
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded IMAGE: 1
@query_time: 0.03620741702616215
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded IMAGE: 1
@query_time: 0.02578231692314148
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded IMAGE: 1
@query_time: 0.020218927413225174
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded IMAGE: 1
@query_time: 0.02314240299165249
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of load

### License Plate Recognition

In [8]:
cursor.execute("""CREATE UDF IF NOT EXISTS OCRExtractor
                  INPUT  (frame NDARRAY UINT8(3, ANYDIM, ANYDIM))
                  OUTPUT (labels NDARRAY STR(10),
                          bboxes NDARRAY FLOAT32(ANYDIM, 4),
                          scores NDARRAY FLOAT32(ANYDIM))
                  TYPE  OCRExtraction
                  IMPL  '../eva/udfs/ocr_extractor.py' ;
      """)
response = cursor.fetch_all()
print(response)

cursor.execute("""CREATE UDF IF NOT EXISTS FuzzDistance
                    INPUT (Input_Array1 NDARRAY ANYTYPE, Input_Array2 NDARRAY ANYTYPE)
                    OUTPUT (distance FLOAT(32, 7))
                    TYPE NdarrayUDF
                    IMPL "../eva/udfs/ndarray/fuzzy_join.py";
                    """)
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                                       0
0  UDF OCRExtractor successfully added to the database.
@query_time: 2.5161344539374113
@status: ResponseStatus.SUCCESS
@batch: 
                                                       0
0  UDF FuzzDistance successfully added to the database.
@query_time: 0.02586629055440426


In [9]:
cursor.execute("CREATE TABLE IF NOT EXISTS LicensePlateCSV(id INTEGER UNIQUE, label TEXT(30));")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
 Empty DataFrame
Columns: []
Index: []
@query_time: 0.041497599333524704


In [10]:
cursor.execute("LOAD CSV 'data.csv' INTO LicensePlateCSV;")
response = cursor.fetch_all()
print(response)


@status: ResponseStatus.SUCCESS
@batch: 
         CSV  Number of loaded frames
0  data.csv                        5
@query_time: 0.035665012896060944


In [11]:
cursor.execute('SELECT OCRExtractor(data) FROM MyImages;')
res = cursor.fetch_all()
print(res)

@status: ResponseStatus.SUCCESS
@batch: 
             ocrextractor.labels  \
0                  [KLL1CA2555]   
1          [Coi, rrm, PGoMN112]   
2                    [PRErINLB]   
3                    [DZIZ YXR]   
4                  [pPuie, BES]   
5      [Wet, alamy stock photo]   
6  [802.LIN, Fay, VIRGINIA, 07]   

                                                                                   ocrextractor.bboxes  \
0                                                   [[[241, 117], [424, 117], [424, 178], [241, 178]]]   
1  [[[320, 96], [342, 96], [342, 104], [320, 104]], [[56, 100], [90, 100], [90, 108], [56, 108]], [...   
2                                                   [[[227, 173], [269, 173], [269, 193], [227, 193]]]   
3                                                   [[[150, 128], [250, 128], [250, 158], [150, 158]]]   
4  [[[175.83355283634924, 85.05243648347647], [388.302989236221, 113.80042194340535], [367.16644716...   
5   [[[362, 274], [384, 274], [384, 280],

In [14]:
cursor.execute('SELECT OCRExtractor(data) FROM MyImages;')
res = cursor.fetch_all()
print(res)

@status: ResponseStatus.FAIL
@batch: 
 None
@error: module must have its parameters and buffers on device cuda:0 (device_ids[0]) but found one of them on device: cuda:1


In [13]:
cursor.execute('SELECT * FROM MyImages JOIN LATERAL OCRExtractor(data) AS T(a,b,c) JOIN LicensePlateCSV B ON FuzzDistance(T.a, B.label) > 50;')
res = cursor.fetch_all()
print(res)


@status: ResponseStatus.FAIL
@batch: 
 None
@error: module must have its parameters and buffers on device cuda:0 (device_ids[0]) but found one of them on device: cuda:1
